In [1]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import os, csv
import seaborn as sns
 
import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### 1. Describe the data gathering process and build dataset from Video

The purpose is to gather data to determine the correct standing posture for Bicep Curl exercise
There are 2 stages:
- Correct: "C"
- Wrong: "W"

In [2]:
mp_pose.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [3]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST"
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

#### 1.2. Set up important functions

In [4]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    

def init_csv(dataset_path: str):
    '''
    Create a blank csv file with just columns
    '''

    # Ignore if file is already exist
    if os.path.exists(dataset_path):
        return

    # Write all the columns to a empty file
    with open(dataset_path, mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(HEADERS)




   
#def export_landmark_to_csv(dataset_path: str, results, action: str):
    '''
    Export labeled data from detected landmarks to CSV (only for selected landmarks).
    '''
    landmarks = results.pose_landmarks.landmark  # Extract all landmarks
    keypoints = []

    try:
        # Extract coordinates of only IMPORTANT_LMS landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
            keypoints.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        # Insert action as the label (first column)
        keypoints.insert(0, action)  

        # Append new row to .csv file
        with open(dataset_path, mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        print(f" Error saving landmarks: {e}")


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)
    

def concat_csv_files_with_same_headers(file_paths: list, saved_path: str):
    '''
    Concat different csv files
    '''
    all_df = []
    for path in file_paths:
        df = pd.read_csv(path, index_col=None, header=0)
        all_df.append(df)
    
    results = pd.concat(all_df, axis=0, ignore_index=True)
    results.to_csv(saved_path, sep=',', encoding='utf-8', index=False)

## Extract data from video

In [5]:
# Paths to the dataset and videos
DATASET_PATH = "train_own.csv"
VIDEO_PATHS = ["correct_posture.mp4", "wrong_posture.mp4",'test_correct.MOV','test_wrong.MOV']
VIDEO_LABELS = ["C", "W","C","W"]  # "C" = Correct, "W" = Incorrect

# Select only important landmarks
IMPORTANT_LMS = [
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST"
]

# Generate headers
HEADERS = ["label"]
for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

# Initialize CSV with headers if not exists
def init_csv(dataset_path):
    if not os.path.exists(dataset_path):
        pd.DataFrame(columns=HEADERS).to_csv(dataset_path, index=False)

# Save only selected landmarks
def export_landmark_to_csv(dataset_path, results, label):
    landmarks = results.pose_landmarks.landmark
    keypoints = []

    try:
        # Extract only important landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[getattr(mp_pose.PoseLandmark, lm).value]
            keypoints.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        # Insert action as label
        keypoints.insert(0, label)

        # Append row to CSV
        df = pd.DataFrame([keypoints], columns=HEADERS)
        df.to_csv(dataset_path, mode='a', header=False, index=False)

    except Exception as e:
        print(f"Error saving landmarks: {e}")

# Initialize CSV
init_csv(DATASET_PATH)

# Process each video
for video_path, label in zip(VIDEO_PATHS, VIDEO_LABELS):
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        continue

    cap = cv2.VideoCapture(video_path)
    save_counts = 0

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            # Resize frame
            image = cv2.resize(image, (640, 480))
            image = cv2.flip(image, 1)

            # Convert BGR to RGB for MediaPipe
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            if not results.pose_landmarks:
                print(f"⚠ Cannot detect pose in {video_path}")
                continue

            # Convert back to BGR for OpenCV
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw landmarks
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Save selected landmarks with label
            export_landmark_to_csv(DATASET_PATH, results, label)
            save_counts += 1

            # Display video with pose landmarks
            cv2.putText(image, f"Saved: {save_counts}", (50, 50),
                        cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow("Pose Detection", image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# Fix MacOS window closing issue
for i in range(1, 5):
    cv2.waitKey(1)

print(f"Dataset saved to {DATASET_PATH}")


⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_posture.mp4
⚠ Cannot detect pose in correct_

In [6]:
# csv_files = [os.path.join("./", f) for f in os.listdir("./") if "csv" in f]

# concat_csv_files_with_same_headers(csv_files, "train.csv")

df = pd.read_csv("train_own.csv")
df

,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,right_elbow_x,...,left_elbow_z,left_elbow_v,right_wrist_x,right_wrist_y,right_wrist_z,right_wrist_v,left_wrist_x,left_wrist_y,left_wrist_z,left_wrist_v
0,C,-0.044216,0.386231,0.411164,0.982325,-0.143734,0.277283,-0.483864,0.998737,-0.276533,...,0.494320,0.037879,-0.311754,0.842192,-0.535266,0.598057,-0.091958,0.836620,0.346546,0.054271
1,C,-0.034352,0.398166,0.408753,0.983107,-0.138219,0.302271,-0.426824,0.998779,-0.276540,...,0.492486,0.038720,-0.306752,0.852927,-0.471621,0.601852,-0.087764,0.871871,0.360655,0.053712
2,C,-0.028611,0.416352,0.423748,0.984051,-0.054798,0.379504,-0.476311,0.998789,-0.147794,...,0.492428,0.041030,-0.112071,1.009123,-0.363381,0.582884,-0.033515,0.905120,0.348712,0.058386
3,C,-0.022435,0.420695,0.427159,0.984920,-0.030279,0.381017,-0.480623,0.998853,-0.117189,...,0.480123,0.041489,-0.083852,1.032205,-0.381322,0.569298,-0.045217,0.935300,0.334156,0.059308
4,C,-0.018270,0.419690,0.497936,0.984963,-0.018456,0.370617,-0.511962,0.998890,-0.124056,...,0.528991,0.039624,-0.086611,1.017712,-0.419686,0.551223,-0.049895,0.960669,0.362218,0.055396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10961,W,0.617818,0.550781,-0.550077,0.999774,0.341379,0.555138,-0.514241,0.999863,0.254916,...,-0.365748,0.996947,0.258317,0.872327,-0.304974,0.897586,0.758564,0.845658,-0.311104,0.954562
10962,W,0.615154,0.552717,-0.550908,0.999768,0.339763,0.557875,-0.512632,0.999855,0.254537,...,-0.385472,0.997028,0.258682,0.872353,-0.517514,0.898985,0.751107,0.847455,-0.377887,0.955381
10963,W,0.613247,0.553775,-0.551805,0.999759,0.338416,0.560067,-0.516488,0.999847,0.254804,...,-0.387242,0.997050,0.261962,0.872356,-0.445550,0.896736,0.745473,0.852024,-0.361914,0.955175
10964,W,0.612171,0.554234,-0.546462,0.999750,0.337080,0.562646,-0.522205,0.999831,0.255573,...,-0.382148,0.997040,0.266955,0.872395,-0.491830,0.892921,0.740711,0.854995,-0.369187,0.955378


### 3. Clean Data and Visualize data

In [7]:
remove_duplicate_rows("./train_own.csv")

### 4. Gather Test Dataset

In [8]:
df1=pd.read_csv('cleaned_train.csv')
len(df)

10966

## Creating the model

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix, roc_curve, auc,classification_report

## Describing the dataset and splitting

In [10]:

# Categorizing label
df1.loc[df1["label"] == "C", "label"] = 0 # Correct 0, wrong 1
df1.loc[df1["label"] == "W", "label"] = 1

In [11]:
df1.tail()

,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,right_elbow_x,...,left_elbow_z,left_elbow_v,right_wrist_x,right_wrist_y,right_wrist_z,right_wrist_v,left_wrist_x,left_wrist_y,left_wrist_z,left_wrist_v
10961,1,0.617818,0.550781,-0.550077,0.999774,0.341379,0.555138,-0.514241,0.999863,0.254916,...,-0.365748,0.996947,0.258317,0.872327,-0.304974,0.897586,0.758564,0.845658,-0.311104,0.954562
10962,1,0.615154,0.552717,-0.550908,0.999768,0.339763,0.557875,-0.512632,0.999855,0.254537,...,-0.385472,0.997028,0.258682,0.872353,-0.517514,0.898985,0.751107,0.847455,-0.377887,0.955381
10963,1,0.613247,0.553775,-0.551805,0.999759,0.338416,0.560067,-0.516488,0.999847,0.254804,...,-0.387242,0.997050,0.261962,0.872356,-0.445550,0.896736,0.745473,0.852024,-0.361914,0.955175
10964,1,0.612171,0.554234,-0.546462,0.999750,0.337080,0.562646,-0.522205,0.999831,0.255573,...,-0.382148,0.997040,0.266955,0.872395,-0.491830,0.892921,0.740711,0.854995,-0.369187,0.955378
10965,1,0.612197,0.555190,-0.535091,0.999753,0.337545,0.562721,-0.512774,0.999829,0.256838,...,-0.375346,0.997113,0.270860,0.872799,-0.433369,0.893592,0.738690,0.857271,-0.363073,0.956998


In [12]:
df1.info

<bound method DataFrame.info of       label  left_shoulder_x  left_shoulder_y  left_shoulder_z  \
0         0        -0.044216         0.386231         0.411164   
1         0        -0.034352         0.398166         0.408753   
2         0        -0.028611         0.416352         0.423748   
3         0        -0.022435         0.420695         0.427159   
4         0        -0.018270         0.419690         0.497936   
...     ...              ...              ...              ...   
10961     1         0.617818         0.550781        -0.550077   
10962     1         0.615154         0.552717        -0.550908   
10963     1         0.613247         0.553775        -0.551805   
10964     1         0.612171         0.554234        -0.546462   
10965     1         0.612197         0.555190        -0.535091   

       left_shoulder_v  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0             0.982325         -0.143734          0.277283         -0.483864   
1             0

In [13]:
df1['label'].astype

<bound method NDFrame.astype of 0        0
1        0
2        0
3        0
4        0
        ..
10961    1
10962    1
10963    1
10964    1
10965    1
Name: label, Length: 10966, dtype: object>

In [14]:
df1['label']=df1['label'].astype(int)

In [15]:
print(df1['label'].sum())

4230


In [16]:
print(len(df1['label']))

10966


In [17]:
sc = StandardScaler()

In [18]:
x = df1.drop("label", axis=1)  # Remove "label"
y = df1["label"].astype('int')  # Convert label to integer type


In [19]:
x = pd.DataFrame(sc.fit_transform(x), columns=x.columns)  # Fit and transform


In [20]:
x.head()

,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,right_elbow_x,right_elbow_y,...,left_elbow_z,left_elbow_v,right_wrist_x,right_wrist_y,right_wrist_z,right_wrist_v,left_wrist_x,left_wrist_y,left_wrist_z,left_wrist_v
0,-8.429205,-1.074621,3.257043,-0.940293,-9.084973,-2.549754,-1.845138,0.018176,-7.152652,-1.662504,...,4.546314,-10.862070,-6.434602,0.456860,-0.233475,-1.623185,-7.001515,0.427567,3.328141,-6.965541
1,-8.303918,-0.900930,3.244164,-0.894857,-8.993462,-2.221583,-1.537965,0.020300,-7.152727,-1.423634,...,4.535033,-10.852318,-6.386111,0.523240,-0.021987,-1.597296,-6.963398,0.618373,3.385771,-6.970141
2,-8.231006,-0.636249,3.324288,-0.839956,-7.609319,-1.207292,-1.804465,0.020856,-5.697830,0.492471,...,4.534681,-10.825554,-4.498835,1.489056,0.337691,-1.726692,-6.470358,0.798347,3.336988,-6.931681
3,-8.152549,-0.573049,3.342515,-0.789495,-7.202506,-1.187423,-1.827685,0.024135,-5.351987,0.577201,...,4.458997,-10.820228,-4.225266,1.631779,0.278072,-1.819373,-6.576716,0.961707,3.277528,-6.924091
4,-8.099650,-0.587671,3.720697,-0.786981,-7.006337,-1.324005,-1.996457,0.026009,-5.429582,0.260856,...,4.759558,-10.841850,-4.252011,1.542163,0.150591,-1.942677,-6.619224,1.099022,3.392157,-6.956280


## Train Test Split and analysis

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)


In [22]:
y_train.sum() # The length of y_traing 8772
# there are 3365 samples of wrong posture, meaning that it has enough of the both postures.


3365

In [23]:
y_train.shape

(8772,)

In [24]:
y_train[:5]

135     0
1300    0
210     0
1564    0
9566    0
Name: label, dtype: int32

In [25]:
y_test.sum() # The length of y_test = 2194,there are 865 samples of wrong posture, meaning that it can test properly wrong and correct postures in similar ratio.


865

## Train and Evaluation

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [35]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []
def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,SVC,"[1.0, 1.0]",1.000000,"[1.0, 1.0]","[1.0, 1.0]","[[1329, 0], [0, 865]]"
1,RF,"[1.0, 0.998]",0.999088,"[0.998, 1.0]","[0.999, 0.999]","[[1327, 2], [0, 865]]"
2,KNN,"[0.998, 0.999]",0.998633,"[0.999, 0.998]","[0.999, 0.998]","[[1328, 1], [2, 863]]"
3,DTC,"[0.998, 0.995]",0.996809,"[0.997, 0.997]","[0.997, 0.996]","[[1325, 4], [3, 862]]"
4,SGDC,"[0.979, 0.991]",0.983592,"[0.994, 0.968]","[0.987, 0.979]","[[1321, 8], [28, 837]]"
5,LR,"[0.979, 0.989]",0.982680,"[0.993, 0.966]","[0.986, 0.978]","[[1320, 9], [29, 836]]"
6,NB,"[0.89, 0.89]",0.889699,"[0.934, 0.822]","[0.911, 0.855]","[[1241, 88], [154, 711]]"
